In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from sklearn.metrics.pairwise import cosine_similarity
from random import sample
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from random import sample
import networkx as nx

from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim

from pandarallel import pandarallel
pandarallel.initialize()
from sklearn.metrics.pairwise import cosine_similarity
from random import sample


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
month = 'july'

framepath = "/data/shruti/ONR/small_data/embeddings/" + month + "_frame.csv"
embedpath = "/data/shruti/ONR/small_data/embeddings/" + month + "_embeddings.pkl"
writepath = "../lite_data/forCliques/" + month + ".gexf"

In [3]:
sample_frame = pd.read_csv(framepath, header=0)

In [4]:
with open(embedpath, "rb") as pfile:
    embeddings = pkl.load(pfile)

In [5]:
embeddings.shape

(136875, 512)

In [6]:
embeddings

array([[-0.01867597,  0.01237004, -0.02475169, ...,  0.03281558,
        -0.03126404,  0.04817518],
       [-0.01867598,  0.01237005, -0.02475168, ...,  0.03281559,
        -0.03126405,  0.04817518],
       [-0.02283544, -0.00666481, -0.01994725, ...,  0.00304299,
         0.00930624, -0.01392937],
       ...,
       [ 0.00173779, -0.00487363,  0.00201711, ...,  0.00081621,
        -0.00213724,  0.00527894],
       [ 0.00904831,  0.03666892, -0.01863558, ...,  0.000504  ,
        -0.01066653, -0.01319246],
       [-0.01355695, -0.02244426, -0.00232942, ...,  0.00194755,
        -0.00123882, -0.00677852]], dtype=float32)

In [7]:
sample_cosSim = cosine_similarity(embeddings)

In [8]:
sample_cosSim

array([[0.9999997 , 0.9999997 , 0.23269069, ..., 0.10383225, 0.11575143,
        0.12173033],
       [0.9999997 , 0.99999964, 0.23269072, ..., 0.10383222, 0.11575145,
        0.1217303 ],
       [0.23269069, 0.23269072, 0.9999999 , ..., 0.04924652, 0.17588782,
        0.10740408],
       ...,
       [0.10383225, 0.10383222, 0.04924652, ..., 0.99999976, 0.03942747,
        0.32937318],
       [0.11575143, 0.11575145, 0.17588782, ..., 0.03942747, 1.0000001 ,
        0.05764032],
       [0.12173033, 0.1217303 , 0.10740408, ..., 0.32937318, 0.05764032,
        0.99999976]], dtype=float32)

In [9]:
sample_cosSim[np.isnan(sample_cosSim)] = 0
sample_cosSim[np.isinf(sample_cosSim)] = 0
np.fill_diagonal(sample_cosSim, 0)
sample_cosSim[sample_cosSim < 0.9] = 0

In [10]:
sample_cosSim

array([[0.       , 0.9999997, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.9999997, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [11]:
nonzero_entries = np.nonzero(sample_cosSim)
nonzero_entries


(array([     0,      1,      2, ..., 136872, 136872, 136872]),
 array([     1,      0,     11, ..., 123631, 123632, 132439]))

In [12]:
tot_nonzero = nonzero_entries[0].shape[0]
tot_nonzero

590966

In [13]:
edges = []
for it in range(tot_nonzero):
    tup = sorted([nonzero_entries[0][it], nonzero_entries[1][it]])
    edges.append((tup[0], tup[1]))


In [14]:
edges[89]

(21, 123353)

In [15]:
edges = list(set(edges))
len(edges)

295483

In [16]:
G = nx.Graph()

for e in edges:
    G.add_edge(e[0], e[1])

In [17]:
ndegree = nx.degree(G)

In [18]:
ndegree

DegreeView({50188: 11, 51394: 23, 15359: 442, 36816: 329, 19109: 26, 20125: 25, 31141: 23, 54927: 23, 57941: 215, 92020: 167, 20846: 183, 107458: 178, 40367: 154, 50625: 154, 124888: 3, 124912: 3, 84882: 80, 98012: 80, 37505: 2, 37950: 2, 109542: 8, 109583: 8, 134467: 21, 134469: 21, 84691: 9, 86366: 9, 63044: 346, 127209: 106, 87871: 140, 109681: 271, 35762: 209, 61325: 144, 44105: 207, 56905: 345, 35644: 239, 73347: 188, 74405: 37, 106903: 37, 74801: 37, 113245: 9, 113425: 9, 28569: 80, 71280: 80, 49392: 3, 49393: 3, 79312: 19, 106340: 11, 80242: 170, 80610: 170, 108195: 67, 113710: 350, 62623: 154, 89245: 154, 45539: 190, 52373: 214, 103313: 4, 116023: 4, 92378: 6, 92380: 6, 63711: 372, 64855: 239, 8210: 149, 31952: 141, 105355: 271, 128178: 188, 88962: 18, 95361: 18, 8695: 160, 100641: 160, 3790: 6, 13385: 6, 94865: 4, 96483: 4, 30766: 7, 35428: 7, 12377: 26, 18747: 26, 51754: 151, 60937: 420, 111917: 8, 111940: 8, 132682: 6, 132708: 6, 88020: 20, 97322: 20, 124562: 4, 125277: 4, 7

In [19]:
valid_nodes = []
for k in dict(ndegree).keys():
    if ndegree[k] > 3:
        valid_nodes.append(k)

In [20]:
len(valid_nodes)

27921

In [21]:
invalid_nodes = list(set(list(G.nodes)).difference(set(valid_nodes)))

In [22]:
for i in invalid_nodes:
    G.remove_node(i)

In [23]:
nx.write_gexf(G, writepath, encoding='utf-8')